In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Load results CSV
CSV_FILE = "results_1.3.csv"
res_df = pd.read_csv(CSV_FILE)

# Ensure plots directory exists
os.makedirs('plots', exist_ok=True)

# Normalize dataframe
df = res_df.copy()
df['user'] = df.get('user', 'unknown')
df['version'] = df['version'].astype(str).str.strip().str.lower()
df['size'] = pd.to_numeric(df.get('size'), errors='coerce').fillna(0).astype(int)

time_cols = ["time_init", "time_compute", "time_thread_create", "time_thread_join", "time_cleanup", "time_total"]
for col in time_cols:
    df[col] = pd.to_numeric(df.get(col), errors='coerce').fillna(0)

unique_users = sorted(df["user"].unique())
unique_sizes = sorted(df["size"].unique())
versions = ["sequential", "parallel_unpadded", "parallel_padded", "parallel_local_accum"]
colors = {
    "sequential": "#7f7f7f",
    "parallel_unpadded": "#ff7f0e",
    "parallel_padded": "#2ca02c",
    "parallel_local_accum": "#d62728"
}

# Track saved files to avoid double-saving within one run
_saved_plot_paths = set()

def size_label(n):
    return f"{n//1_000_000}M" if n >= 1_000_000 else f"{n//1000}K" if n >= 1000 else str(n)

# --- Function to overlay speedups with error propagation ---
def plot_speedup_overlay(seq_means, par_means_dict, sizes, title, outpath, seq_stds=None, par_stds_dict=None):
    fig, ax = plt.subplots(figsize=(12, 6))
    fig.suptitle(title, fontsize=14, fontweight="bold")

    x = np.arange(len(sizes))
    for par_version, par_means in par_means_dict.items():
        # Convert to numpy arrays
        seq = np.asarray(seq_means, dtype=float)
        par = np.asarray(par_means, dtype=float)

        with np.errstate(divide='ignore', invalid='ignore'):
            speedup = np.where(par > 0, seq / par, np.nan)

        # Error propagation for S = A / B: var(S) ≈ S^2 * ( (σ_A/μ_A)^2 + (σ_B/μ_B)^2 )
        speedup_err = np.full_like(speedup, np.nan)
        if seq_stds is not None and par_stds_dict is not None and par_version in par_stds_dict:
            seq_s = np.asarray(seq_stds, dtype=float)
            par_s = np.asarray(par_stds_dict[par_version], dtype=float)
            # Avoid division by zero; only compute where means > 0
            valid = (seq > 0) & (par > 0)
            rel_sq = np.zeros_like(speedup)
            # compute relative squared errors safely
            rel_sq[valid] = (seq_s[valid] / seq[valid]) ** 2 + (par_s[valid] / par[valid]) ** 2
            speedup_err[valid] = np.abs(speedup[valid]) * np.sqrt(rel_sq[valid])

        # Plot mean
        ax.plot(x, speedup, marker='o', linewidth=2, markersize=8,
                label=par_version.replace('_', ' ').capitalize(), color=colors.get(par_version, '#444'))

        # Plot error band if available
        if np.any(np.isfinite(speedup_err)):
            lower = speedup - speedup_err
            upper = speedup + speedup_err
            ax.fill_between(x, lower, upper, alpha=0.2, color=colors.get(par_version, '#444'))
            # also show small errorbars on markers
            ax.errorbar(x, speedup, yerr=speedup_err, fmt='none', ecolor=colors.get(par_version, '#444'), capsize=4, alpha=0.8)

    ax.set_xlabel("Array Size", fontsize=11)
    ax.set_ylabel("Speedup", fontsize=11)
    ax.set_xticks(x)
    ax.set_xticklabels([size_label(s) for s in sizes])
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=10)
    plt.tight_layout()

    # Avoid saving same path twice in one run
    if outpath not in _saved_plot_paths:
        fig.savefig(outpath, dpi=300, bbox_inches="tight")
        _saved_plot_paths.add(outpath)
        print(f"Saved: {outpath}")
    else:
        print(f"Skipped duplicate save: {outpath}")

    plt.close(fig)

# --- Per-user plots ---
for user in unique_users:
    user_df = df[df["user"] == user].copy()
    if user_df.empty:
        continue

    stats = user_df.groupby(["version", "size"])[time_cols].agg(["mean", "std"]).reset_index()
    stats.columns = ["_".join(c).strip("_") for c in stats.columns]

    # Timing plots (same as before)
    for metric in ["time_total", "time_compute"]:
        fig, ax = plt.subplots(figsize=(12, 6))
        fig.suptitle(f"Array Analysis — {metric.replace('time_', '').replace('_', ' ').title()} — User: {user}", fontsize=14, fontweight="bold")
        jitter_offset = np.linspace(-0.2, 0.2, len(versions))

        for idx, version in enumerate(versions):
            version_stats = stats[stats["version"] == version]
            if version_stats.empty: continue
            x_jitter = np.arange(len(unique_sizes)) + jitter_offset[idx]
            y_mean = version_stats[f"{metric}_mean"].values
            y_std = version_stats[f"{metric}_std"].values
            ax.errorbar(x_jitter, y_mean, yerr=y_std, marker='o', label=version.replace('_', ' ').capitalize(),
                        linewidth=2, capsize=5, linestyle='-', alpha=0.8, color=colors.get(version, "#444"))

        ax.set_xlabel("Array Size", fontsize=11)
        ax.set_ylabel(f"{metric.replace('time_', '').replace('_', ' ').title()} (s)", fontsize=11)
        ax.set_xticks(range(len(unique_sizes)))
        ax.set_xticklabels([size_label(s) for s in unique_sizes])
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        outpath = f"plots/plot_1_3_{user}_{metric}.png"
        if outpath not in _saved_plot_paths:
            fig.savefig(outpath, dpi=300, bbox_inches="tight")
            _saved_plot_paths.add(outpath)
            print(f"Saved: {outpath}")
        else:
            print(f"Skipped duplicate save: {outpath}")
        plt.close(fig)

    # --- Overlay speedup plot (total) ---
    seq_stats = stats[stats["version"] == "sequential"]
    common_sizes = sorted(set(seq_stats["size"]) & set(stats["size"]))
    if not common_sizes: continue

    seq_times = seq_stats[seq_stats["size"].isin(common_sizes)].set_index("size")["time_total_mean"].reindex(common_sizes).values
    seq_times_std = seq_stats[seq_stats["size"].isin(common_sizes)].set_index("size")["time_total_std"].reindex(common_sizes).values

    par_times_dict = {}
    par_times_std_dict = {}
    for par_version in ["parallel_unpadded", "parallel_padded", "parallel_local_accum"]:
        par_times = stats[(stats["version"] == par_version) & (stats["size"].isin(common_sizes))]
        par_times = par_times.set_index("size")["time_total_mean"].reindex(common_sizes).values
        par_times_std = stats[(stats["version"] == par_version) & (stats["size"].isin(common_sizes))].set_index("size")["time_total_std"].reindex(common_sizes).values
        par_times_dict[par_version] = par_times
        par_times_std_dict[par_version] = par_times_std

    plot_speedup_overlay(seq_times, par_times_dict, common_sizes,
                         f"Array Analysis — Total-time Speedup — User: {user}",
                         f"plots/plot_1_3_{user}_speedup_overlay.png",
                         seq_stds=seq_times_std, par_stds_dict=par_times_std_dict)

    # --- Overlay compute-time speedup for this user ---
    seq_times_compute = seq_stats[seq_stats["size"].isin(common_sizes)].set_index("size")["time_compute_mean"].reindex(common_sizes).values
    seq_times_compute_std = seq_stats[seq_stats["size"].isin(common_sizes)].set_index("size")["time_compute_std"].reindex(common_sizes).values
    par_times_compute_dict = {}
    par_times_compute_std_dict = {}
    for par_version in ["parallel_unpadded", "parallel_padded", "parallel_local_accum"]:
        par_times_c = stats[(stats["version"] == par_version) & (stats["size"].isin(common_sizes))]
        par_times_c_mean = par_times_c.set_index("size")["time_compute_mean"].reindex(common_sizes).values
        par_times_c_std = par_times_c.set_index("size")["time_compute_std"].reindex(common_sizes).values
        par_times_compute_dict[par_version] = par_times_c_mean
        par_times_compute_std_dict[par_version] = par_times_c_std

    plot_speedup_overlay(seq_times_compute, par_times_compute_dict, common_sizes,
                         f"Array Analysis — Compute-time Speedup — User: {user}",
                         f"plots/plot_1_3_{user}_compute_speedup_overlay.png",
                         seq_stds=seq_times_compute_std, par_stds_dict=par_times_compute_std_dict)

# --- All-users combined overlay speedup ---
if len(unique_users) > 1:
    stats_all = df.groupby(["version", "size"])[time_cols].agg(["mean", "std"]).reset_index()
    stats_all.columns = ["_".join(c).strip("_") for c in stats_all.columns]

    common_sizes = sorted(set(stats_all[stats_all["version"] == "sequential"]["size"]) &
                          set(stats_all["size"]))
    seq_times_all = stats_all[(stats_all["version"] == "sequential") & (stats_all["size"].isin(common_sizes))]
    seq_times_all_mean = seq_times_all.set_index("size")["time_total_mean"].reindex(common_sizes).values
    seq_times_all_std = seq_times_all.set_index("size")["time_total_std"].reindex(common_sizes).values

    par_times_dict_all = {}
    par_times_dict_all_std = {}
    for par_version in ["parallel_unpadded", "parallel_padded", "parallel_local_accum"]:
        par_times = stats_all[(stats_all["version"] == par_version) & (stats_all["size"].isin(common_sizes))]
        par_times_mean = par_times.set_index("size")["time_total_mean"].reindex(common_sizes).values
        par_times_std = par_times.set_index("size")["time_total_std"].reindex(common_sizes).values
        par_times_dict_all[par_version] = par_times_mean
        par_times_dict_all_std[par_version] = par_times_std

    plot_speedup_overlay(seq_times_all_mean, par_times_dict_all, common_sizes,
                         "Array Analysis — Total-time Speedup — All Users",
                         "plots/plot_1_3_all_users_speedup_overlay.png",
                         seq_stds=seq_times_all_std, par_stds_dict=par_times_dict_all_std)

    # --- All-users compute-time overlay speedup ---
    seq_times_all_compute = stats_all[(stats_all["version"] == "sequential") & (stats_all["size"].isin(common_sizes))]
    seq_times_all_compute_mean = seq_times_all_compute.set_index("size")["time_compute_mean"].reindex(common_sizes).values
    seq_times_all_compute_std = seq_times_all_compute.set_index("size")["time_compute_std"].reindex(common_sizes).values

    par_times_dict_all_compute = {}
    par_times_dict_all_compute_std = {}
    for par_version in ["parallel_unpadded", "parallel_padded", "parallel_local_accum"]:
        par_times_c = stats_all[(stats_all["version"] == par_version) & (stats_all["size"].isin(common_sizes))]
        par_times_c_mean = par_times_c.set_index("size")["time_compute_mean"].reindex(common_sizes).values
        par_times_c_std = par_times_c.set_index("size")["time_compute_std"].reindex(common_sizes).values
        par_times_dict_all_compute[par_version] = par_times_c_mean
        par_times_dict_all_compute_std[par_version] = par_times_c_std

    plot_speedup_overlay(seq_times_all_compute_mean, par_times_dict_all_compute, common_sizes,
                         "Array Analysis — Compute-time Speedup — All Users",
                         "plots/plot_1_3_all_users_compute_speedup_overlay.png",
                         seq_stds=seq_times_all_compute_std, par_stds_dict=par_times_dict_all_compute_std)
else:
    print(f"Skipping all-users overlay generation: only {len(unique_users)} user(s) present.")

print("\nOverlay speedup plots generated for individual users (total + compute).")
if len(unique_users) > 1:
    print("All-users overlays generated (total + compute).")

Saved: plots/plot_1_3_ea24205_time_total.png
Saved: plots/plot_1_3_ea24205_time_compute.png
Saved: plots/plot_1_3_ea24205_speedup_overlay.png
Saved: plots/plot_1_3_ea24205_compute_speedup_overlay.png
Saved: plots/plot_1_3_marr_time_total.png
Saved: plots/plot_1_3_marr_time_compute.png
Saved: plots/plot_1_3_marr_speedup_overlay.png
Saved: plots/plot_1_3_marr_compute_speedup_overlay.png
Saved: plots/plot_1_3_phoebus_time_total.png
Saved: plots/plot_1_3_phoebus_time_compute.png
Saved: plots/plot_1_3_phoebus_speedup_overlay.png
Saved: plots/plot_1_3_phoebus_compute_speedup_overlay.png
Saved: plots/plot_1_3_all_users_speedup_overlay.png
Saved: plots/plot_1_3_all_users_compute_speedup_overlay.png

Overlay speedup plots generated for individual users (total + compute).
All-users overlays generated (total + compute).


In [2]:
# Build a concise summary table per size/version with speedups (and propagated std)
src = stats_all if 'stats_all' in globals() and isinstance(stats_all, pd.DataFrame) and not stats_all.empty else stats

rows = []
for s in sorted(src['size'].unique()):
    seq_row = src[(src['version'] == 'sequential') & (src['size'] == s)]
    seq_total_mean = float(seq_row['time_total_mean']) if not seq_row.empty else np.nan
    seq_total_std = float(seq_row['time_total_std']) if not seq_row.empty else np.nan
    seq_compute_mean = float(seq_row['time_compute_mean']) if not seq_row.empty else np.nan
    seq_compute_std = float(seq_row['time_compute_std']) if not seq_row.empty else np.nan

    for v in sorted(src['version'].unique()):
        r = src[(src['version'] == v) & (src['size'] == s)]
        if r.empty:
            continue
        total_mean = float(r['time_total_mean'])
        total_std = float(r['time_total_std'])
        compute_mean = float(r['time_compute_mean'])
        compute_std = float(r['time_compute_std'])

        # speedup = sequential / current
        def compute_speedup(mu_a, sigma_a, mu_b, sigma_b):
            if mu_a > 0 and mu_b > 0:
                s = mu_a / mu_b
                rel_sq = (sigma_a / mu_a) ** 2 + (sigma_b / mu_b) ** 2
                s_std = abs(s) * np.sqrt(rel_sq)
                return s, s_std
            return np.nan, np.nan

        speedup_total_mean, speedup_total_std = compute_speedup(seq_total_mean, seq_total_std, total_mean, total_std)
        speedup_compute_mean, speedup_compute_std = compute_speedup(seq_compute_mean, seq_compute_std, compute_mean, compute_std)

        rows.append({
            'size': s,
            'version': v,
            'time_total_mean_ms': total_mean * 1000,
            'time_total_std_ms': total_std * 1000,
            'time_compute_mean_ms': compute_mean * 1000,
            'time_compute_std_ms': compute_std * 1000,
            'speedup_total_percentage': speedup_total_mean * 100,
            'speedup_total_std_percentage': speedup_total_std * 100,
            'speedup_compute_percentage': speedup_compute_mean * 100,
            'speedup_compute_std_percentage': speedup_compute_std * 100
        })

summary_df = pd.DataFrame(rows).sort_values(['size', 'version']).reset_index(drop=True)

# nicer formatting for display in Jupyter
pd.set_option('display.precision', 4)
display(summary_df)

# # save CSV for later reference
# out_csv = "plots/summary_table.csv"
# summary_df.to_csv(out_csv, index=False)
# print(f"Saved summary to: {out_csv}")

C:\Users\mrlnp\AppData\Local\Temp\ipykernel_20276\2838813720.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  seq_total_mean = float(seq_row['time_total_mean']) if not seq_row.empty else np.nan
C:\Users\mrlnp\AppData\Local\Temp\ipykernel_20276\2838813720.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  seq_total_std = float(seq_row['time_total_std']) if not seq_row.empty else np.nan
C:\Users\mrlnp\AppData\Local\Temp\ipykernel_20276\2838813720.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  seq_compute_mean = float(seq_row['time_compute_mean']) if not seq_row.empty else np.nan
C:\Users\mrlnp\AppData\Local\Temp\ipykernel_20276\2838813720.py:10: FutureWarning: Calling float on a single e

,size,version,time_total_mean_ms,time_total_std_ms,time_compute_mean_ms,time_compute_std_ms,speedup_total_percentage,speedup_total_std_percentage,speedup_compute_percentage,speedup_compute_std_percentage
0,100000,parallel_local_accum,2.9989,2.1132,0.7913,1.0692,65.8723,69.7568,33.7097,52.3352
1,100000,parallel_padded,4.4624,5.5983,2.0084,4.4614,44.2692,65.6432,13.2809,31.1999
2,100000,parallel_unpadded,2.8910,1.9738,0.8397,0.3434,68.3316,71.3735,31.7641,27.5407
3,100000,sequential,1.9755,1.5616,0.2667,0.2039,100.0000,111.7925,100.0000,108.1205
4,1000000,parallel_local_accum,15.7096,6.8587,1.9596,2.2392,132.3199,101.8650,121.0859,148.5400
5,1000000,parallel_padded,14.3541,5.0727,2.3433,1.1492,144.8156,105.1217,101.2575,67.1410
6,1000000,parallel_unpadded,15.8176,5.2290,3.3926,1.6029,131.4165,93.9719,69.9405,45.4563
7,1000000,sequential,20.7869,13.1803,2.3728,1.0590,100.0000,89.6704,100.0000,63.1146
8,10000000,parallel_local_accum,112.1216,33.1869,10.0577,2.3985,99.8019,35.3239,175.8806,48.6902
9,10000000,parallel_padded,120.3936,33.8302,21.8084,9.2001,92.9447,31.7406,81.1137,36.0692
